<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Simple-Linear-Regression" data-toc-modified-id="Simple-Linear-Regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Simple Linear Regression</a></span></li><li><span><a href="#Multiple-Regression" data-toc-modified-id="Multiple-Regression-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Multiple Regression</a></span></li><li><span><a href="#Nonlinear-Regression" data-toc-modified-id="Nonlinear-Regression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Nonlinear Regression</a></span></li></ul></div>

In [ ]:
import os
from pathlib import Path
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline
from datetime import datetime




In [ ]:
pwd

In [ ]:
vti_df = pd.read_csv('C:/Users/624013/Documents/TEDS/VTI.csv')

In [ ]:
vti_df.info()

In [ ]:
vti_df.info()

In [ ]:
print(type(vti_df))  #<- a Pandas DataFrame
print(len(vti_df))   #<- returns the number of rows

# Simple Linear Regression

In [ ]:
#Simple linear regression

# Subset the two variables for simple linear regression.
df_subset = vti_df[['Adj Close', 'Date']]

In [ ]:
#Summary statistics: dependent and independent variables 

# Describe the `change` summary statistics.
print(df_subset['Adj Close'].describe())
# Describe the `'calories'` summary statistics.
print(df_subset['Date'].describe())

In [ ]:
#Summary statistics: covariance in Python

print(df_subset.cov())


In [ ]:
#Summary statistics: correlation (scaled cov) 

df_subset.corr()

In [ ]:
# EDA: scatter plot  

plt.scatter(df_subset['Adj Close'],
            df_subset['Date'])
plt.title('Adj Close' + " vs Date")
plt.xlabel('Date')
plt.ylabel("Adj Close")
plt.show()


In [ ]:
#EDA: histogram - `Adj Close`  

plt.hist(df_subset['Adj Close'], bins = 10)
plt.xlabel('Adj Close')


In [ ]:
plt.hist(df_subset['Volume'], bins = 10)
plt.xlabel('Volume')

In [ ]:
#Data cleaning: NAs 

# Check how many values are null in the `change` column.
print(df_subset['Adj Close'].isnull().sum())
# Check how many values are null in the `'calories'` column.
print(df_subset['Volume'].isnull().sum())


In [ ]:
#Data cleaning: NAs 

print(df_subset[df_subset['Adj Close'].isnull()])




In [ ]:
#Data cleaning: use fillna()

# Set the dataframe equal to the imputed dataset.
df_subset = df_subset.fillna(df_subset.mean())

# Check how many values are null in the `change` column.
print(df_subset['Adj Close'].isnull().sum())

In [ ]:
#Data cleaning: testing for near zero variance  

# Using sklearn, look for low variance within the columns.
# First, we'll instantiate the function.
selector = VarianceThreshold()

# Next, name the cleaned dataset df_subset_clean.
df_subset_clean = selector.fit_transform(df_subset)

# Let's see if the dimensions changed.
print(df_subset_clean.shape)

In [ ]:
#Implement linear regression 

# Two variables for single regression.
X = pd.DataFrame(df_subset_clean[:,1]) # independent variable

# Make sure to add a constant term so that we have a column for the intercept.
X = sm.add_constant(X)
y = pd.DataFrame(df_subset_clean[:,0])  # dependent variable

In [ ]:
# Set the seed 

# Set the random seed equal to 1.
np.random.seed(1)

In [ ]:
# Implement: build a linear model 

# Build the model, note the difference in argument order.
model = sm.OLS(y, X).fit()


In [ ]:
#Implement: build a linear model

# Inspect the output of the `sm.OLS` function.
print(model.summary())

In [ ]:
#Implement: plot the fit 

# Code to plot using matlplotlib.pyplot and statsmodel abline_plot.
fig = sm.graphics.abline_plot(model_results = model)
ax = fig.axes[0]
plt.scatter(df_subset['Volume'],
            df_subset['Adj Close'])
plt.show()


In [ ]:
# Evaluate: residual standard error  

# Residual standard error
print(np.sqrt(model.scale))

In [ ]:
#Plotting the residuals 

fitted = model.fittedvalues
print(fitted.head())
residuals = model.resid
print(residuals.head())

In [ ]:
#Check for linearity

import seaborn as sns

plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)

plot_lm_1.axes[0] = sns.residplot(fitted, 'Adj Close', 
                    data=df_subset, 
                    lowess = True, 
                    scatter_kws = {'alpha': 0.5}, 
                    line_kws = {'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')

In [ ]:
#Check for normal distribution 

from statsmodels.graphics.gofplots import ProbPlot

model_norm_residuals = model.get_influence().resid_studentized_internal
QQ = ProbPlot(model_norm_residuals)
plot_lm_2 = QQ.qqplot(line= '45', alpha = 0.5, color = '#4C72B0', lw = 1)

plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0].set_title('Normal Q-Q')
plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
plot_lm_2.axes[0].set_ylabel('Standardized Residuals')

In [ ]:
# Check for equal variance 

model_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_norm_residuals))
plot_lm_3 = plt.figure(3)
plot_lm_3.set_figheight(8)
plot_lm_3.set_figwidth(12)

plt.scatter(fitted, model_norm_residuals_abs_sqrt, alpha = 0.5)
sns.regplot(fitted, model_norm_residuals_abs_sqrt,
            scatter = False, 
            ci = False, 
            lowess = True,
            line_kws = {'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('sqrt(|Standardized Residuals|)')


# Multiple Regression

In [ ]:
#### Slide 13/31: Subset data  ####
all_features = ['Open', 'High', 'Low', 'Volume']
all_features.append("Adj Close")

df_subset = vti_df[all_features]
df_subset.head()


In [ ]:
#=================================================-
#### Slide 17/31: Summary statistics: describe the dataset  ####

# Let's look at the summary statistics.
print(df_subset.describe())

In [ ]:
#Summary statistics: covariance
print(df_subset.cov())

In [ ]:
#Summary statistics: correlation
print(df_subset.corr())

In [ ]:
#Histogram of the target variable
plt.hist(df_subset["Adj Close"],  bins = 10)
plt.xlabel("Adj Close")

In [ ]:
# Make scatter plot.
scatter_m = scatter_matrix(df_subset)
plt.show()

In [ ]:
# Check how many values are null in the dataset.
print(df_subset.isnull().sum())
# Drop NA's from the data
df_subset = df_subset.dropna()


In [ ]:
#### Slide 27/31: Data cleaning: near-zero variance  ####

# Using sklearn, let's look for low variance within the columns.
# First, instantiate the function.
selector = VarianceThreshold()
# Then, name the cleaned dataset df_subset_clean.
df_subset_clean = selector.fit_transform(df_subset)
# Let's see if the dimensions changed.
print(df_subset_clean.shape)
print(df_subset_clean)

In [ ]:
#### Data cleaning: near-zero variance  ####

# Let's convert the numpy array back to a DataFrame for convenience.
df_subset_clean = pd.DataFrame(df_subset_clean, columns = all_features)
df_subset_clean.head()

In [ ]:
# Set X to ['Open', 'High', 'Low', 'Volume'].  
X = df_subset_clean[['Open', 'High', 'Low', 'Volume']]
# Add a constant.
X = sm.add_constant(X)

# Set y to `Adj Close`.
y = df_subset_clean[["Adj Close"]]

In [ ]:
#### Apply regression to the dataset  ####

# Set the seed.
np.random.seed(1)

# Create the train and test sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

# Check to see if the datasets split correctly.
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


In [ ]:
####  Multiple linear regression on a dataset  ####

# Build a linear model on training data.
model_m = sm.OLS(y_train, X_train).fit()

In [ ]:
####  Assumptions: plot  ####

fitted_m = model_m.fittedvalues
print(fitted_m.head())
residuals_m = model_m.resid
print(residuals_m.head())



In [ ]:
#### Assumptions: plot (cont’d.)  ####

# Get the normalized residuals.
model_m_norm_residuals = model_m.get_influence().resid_studentized_internal
# Get the absolute squared normalized residuals.
model_m_norm_residuals_abs_sqrt = np.sqrt(np.abs(model_m_norm_residuals))
# Get the absolute residuals. 
model_m_abs_resid = np.abs(residuals_m)
# Combine X_train and y_train into one dataframe for plotting.
frames = [X_train,y_train]
training = pd.concat(frames, axis = 1) # axis = 1 allows us to combine by columns


In [ ]:
####  Assumption: residuals vs. fitted   ####

import seaborn as sns
# Let's look at assumption 1.
plot_lm_1 = plt.figure(1)
plot_lm_1.set_figheight(8)
plot_lm_1.set_figwidth(12)
plot_lm_1.axes[0] = sns.residplot(fitted_m, "Adj Close", data = training, 
                          lowess = True, 
                          scatter_kws = {'alpha': 0.5}, 
                          line_kws = {'color': 'red', 'lw': 1, 'alpha': 0.8})
plot_lm_1.axes[0].set_title('Residuals vs Fitted')
plot_lm_1.axes[0].set_xlabel('Fitted values')
plot_lm_1.axes[0].set_ylabel('Residuals')


In [ ]:
#### Assumption: normally distributed residuals  ####

from statsmodels.graphics.gofplots import ProbPlot

QQ = ProbPlot(model_m_norm_residuals)
plot_lm_2 = QQ.qqplot(line = '45', alpha = 0.5, color = '#4C72B0', lw = 1)

plot_lm_2.set_figheight(8)
plot_lm_2.set_figwidth(12)

plot_lm_2.axes[0].set_title('Normal Q-Q')
plot_lm_2.axes[0].set_xlabel('Theoretical Quantiles')
plot_lm_2.axes[0].set_ylabel('Standardized Residuals');


In [ ]:
#### Slide 26/30: Assumption: equal residual variance  ####

plot_lm_3 = plt.figure(3)
plot_lm_3.set_figheight(8)
plot_lm_3.set_figwidth(12)

plt.scatter(fitted_m, model_m_norm_residuals_abs_sqrt, alpha = 0.5)
sns.regplot(fitted_m, model_m_norm_residuals_abs_sqrt, 
            scatter = False, 
            ci = False, 
            lowess = True,
            line_kws = {'color': 'red', 'lw': 1, 'alpha': 0.8})

plot_lm_3.axes[0].set_title('Scale-Location')
plot_lm_3.axes[0].set_xlabel('Fitted values')
plot_lm_3.axes[0].set_ylabel('$\sqrt{|Standardized Residuals|}$')

In [ ]:
####  Influential cases: residuals vs. leverage  ####

# Identify the influential points ('\n' syntax creates a new line in the output).
test_m = model_m.outlier_test()
print("Bad data points (bonf(p) < 0.05):\n", test_m[test_m['bonf(p)'] < 0.05])
# Save the final outliers.
test_final_m = test_m[test_m['bonf(p)'] < 0.05]

In [ ]:
####  Removing outliers from regression dataset  ####

test_final_m = test_m[test_m['bonf(p)'] < 0.05]
# Make sure that you drop outliers from both X and y train sets.
X_train_no_outliers = X_train.drop(test_final_m.index)
y_train_no_outliers = y_train.drop(test_final_m.index)
# Look at the shape of the new DataFrame to check that the rows have actually been dropped.
print(X_train_no_outliers.shape)
print(y_train_no_outliers.shape)

In [ ]:
#### Rerun multiple regression model  ####

# Build a linear model on training data.
model_m_no_outliers = sm.OLS(y_train_no_outliers, X_train_no_outliers).fit()

In [ ]:
#### Testing the model  ####

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X_train_no_outliers.values, i) for i in range(X_train_no_outliers.shape[1])]
vif["features"] = X_train_no_outliers.columns
print(vif)


In [ ]:
#### Testing the model (cont'd)  ####

if vif[vif['VIF Factor'] > 10].features.shape[0] > 0:
    print("Multicollinearity exists in our model")
else:
    print("No multicollinearity exists in our model")


In [ ]:
#### Predict: `Adj Close` in test data  ####

# Predict values of `Calories` using the test data.
prediction = model_m_no_outliers.predict(X_test)
print(prediction.head())

In [ ]:
#### Slide 18/24: Predict: residuals of model  ####

actual = y_test["Adj Close"]
prediction = model_m_no_outliers.predict(X_test)
residuals = y_test["Adj Close"] - prediction


results =  pd.concat([actual.rename('actual'),
                              prediction.rename('predicted'),
                              residuals.rename('residuals')], axis = 1)
print(results.head())

In [ ]:
#### redict: mean squared error  ####

def rmse(predictions,actual):
    return np.sqrt(((prediction-actual) ** 2).mean())

print(rmse(prediction,actual))

# Nonlinear Regression

In [ ]:
#Slide 10: Transforming target to log 
vti_df['Adj Close_log'] = np.log(vti_df['Adj Close'])
vti_df.head(3)

In [ ]:
# Fitting the log-lin model  

vti_df = sm.add_constant(vti_df)
model_lin = sm.OLS(vti_df['Adj Close_log'], vti_df.loc[:,['const','Volume']]).fit()

In [ ]:
prediction_lin = model_lin.predict(vti_df.loc[:,['const','Volume']])
prediction_lin[1:10]

In [ ]:
prediction = np.exp(prediction_lin)
prediction[1:5]

In [ ]:
#### Slide 14: Evaluating the log-lin model: chart  ####

plt.scatter(vti_df['Volume'],vti_df['Adj Close'])
plt.plot(vti_df['Volume'], prediction, 'red')
plt.title("Index price over time")
plt.xlabel("Days")
plt.ylabel("Index price")
plt.show()